In [1]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from itertools import product
import warnings
from sklearn.metrics import *

warnings.filterwarnings('ignore') # uyarıları kapatır

%matplotlib inline # grafikleri notebook üzerinde gösterir

macro_data = pd.read_csv("C:/users/FurkanBulut/Desktop/T1totalclean.csv", usecols=['Name', 'T3', 'RH3', 'P2', 'WS101']) # veri setini okur
forecast = [] # tahminleri tutar

for i in range(0, 144, 2): # 2 saatlik aralıklarla tahmin yapar
    data = macro_data[i:i+144] # 2 saatlik veri setini alır

    def optimize_SARIMAX(endog, exog, parameters_list, d, D, s): # parametreleri optimize eder
        results = [] # sonuçları tutar
        for param in tqdm_notebook(parameters_list): # parametreleri döndürür
            try:
                model = SARIMAX(endog, 
                                exog,
                                order=(param[0], d, param[1]),
                                seasonal_order=(param[2], D, param[3]),
                                simple_differencing=False).fit(disp=False) # modeli kurar
            except:
                continue
            aic = model.aic # aic değerini alır
            results.append([param, aic]) # sonuçları listeye ekler

        result_df = pd.DataFrame(results) # sonuçları dataframe'e çevirir
        result_df.columns = ['(p,q)x(P,Q)', 'AIC'] # sütun isimlerini değiştirir
        result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True) # aic değerlerine göre sıralar
        return result_df

    p = range(0, 3, 1)
    d = 1
    q = range(0, 3, 1)
    P = range(0, 3, 1)
    D = 0
    Q = range(0, 3, 1)
    s = 4
    parameters = product(p, q, P, Q)
    parameters_list = list(parameters)
    endog = macro_data['WS101'][i:i+142]
    exog = macro_data[['T3', 'RH3', 'P2']][i:i+142]
    result_df = optimize_SARIMAX(endog, exog, parameters_list, 1, 0, 4)
    result_df_min = result_df[result_df.AIC == result_df.AIC.min()]
    p = result_df_min['(p,q)x(P,Q)'][0][0]
    q = result_df_min['(p,q)x(P,Q)'][0][1]
    P = result_df_min['(p,q)x(P,Q)'][0][2]
    Q = result_df_min['(p,q)x(P,Q)'][0][3]
    best_model = SARIMAX(endog,
                        exog,
                        order=(p, d, q),
                        seasonal_order=(P, D, Q, s),
                        simple_differencing=False)
    res = best_model.fit(dis=False)
    n_forecast = 2
    predict = res.get_prediction(end=best_model.nobs + n_forecast, exog=exog.iloc[-3:])
    prediction = predict.predicted_mean[-n_forecast:]
    forecast.append(prediction)

df = macro_data['WS101']
expected = df[144:57601]
prediction = pd.DataFrame(np.concatenate(forecast))
MSE = mean_squared_error(expected, prediction)
R2 = r2_score(expected, prediction)
RMSE = sqrt(mean_squared_error(expected, prediction))
MAE = mean_absolute_error(expected, prediction)

print("MSE:", MSE)
print("R2:", R2)
print("RMSE:", RMSE)
print("MAE:", MAE)


SyntaxError: invalid syntax (741434704.py, line 1)